In [1]:
%load_ext autoreload

In [2]:
%autoreload 2 

In [3]:
%matplotlib inline

In [4]:
import pathlib

In [5]:
import sys

In [6]:
sys.path.append('../../ml4seas/')

In [7]:
import numpy as np 
import pandas as pd 
import xarray as xr

In [8]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K

In [9]:
from NN import *

In [10]:
dpath = pathlib.Path('../../../data/GCMs/processed/hindcasts/CDS/CMCC/T2M/')

In [11]:
lfiles = list(dpath.glob("CMCC_T2M_seasonal_anomalies_????_??.nc"))

In [12]:
lfiles.sort()

In [13]:
dset_train = xr.open_mfdataset(lfiles, concat_dim='time', combine='nested', parallel=True)

In [14]:
dset_train

,Array,Chunk
Bytes,8.96 GB,31.10 MB
Shape,"(288, 40, 3, 180, 360)","(1, 40, 3, 180, 360)"
Count,1152 Tasks,288 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,6.91 kB,24 B
Shape,"(288, 3)","(1, 3)"
Count,1152 Tasks,288 Chunks
Type,datetime64[ns],numpy.ndarray


### select one step, and only t2m 

In [15]:
dset_train = dset_train[['t2m']].sel(step=3)

In [16]:
dset_train

,Array,Chunk
Bytes,2.99 GB,10.37 MB
Shape,"(288, 40, 180, 360)","(1, 40, 180, 360)"
Count,1440 Tasks,288 Chunks
Type,float32,numpy.ndarray


### concatenate all the members long the first dimension, to increase number of instances 

In [17]:
dset_train = dset_train.stack(instance=('time','member'))

In [18]:
dset_train = dset_train.transpose('instance','lat','lon')

In [19]:
batch_size=32

In [20]:
data_train = XrDataGenerator(dset_train, dset_train, {'t2m':None}, 't2m', norm=True, batch_size=batch_size, mean=None, std=None, shuffle=True, load=False)

In [21]:
data_train[0][0].shape

(32, 180, 360, 1)

In [22]:
# Input placeholder
original = tf.keras.layers.Input(shape=(180,360,1))

# Resize to have dimesions divisible by 8
resized = ResizeLayer(newsize=(176,360))(original)

# # Wrap-around in longitude for periodic boundary conditions
# padded = LonPadLayer(padding=8)(resized)

padded = PeriodicPadding2D(8)(resized)

# Encoding layers
x = tf.keras.layers.Conv2D(16, (3, 3), padding='same')(padded)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

In [23]:
# Decoding layers
x = tf.keras.layers.Conv2D(8, (3, 3), padding='same')(encoded)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(8, (3, 3), padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(16, (3, 3), padding='same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), padding='same')(x)

# Strip the longitude wrap-around
pruned=PrunePeriodicPadding2D(8)(decoded)

In [24]:
outsize=ResizeLayer(newsize=(180,360))(pruned)

In [25]:
autoencoder = tf.keras.models.Model(original,outsize)

In [26]:
autoencoder.summary(line_length=120)

Model: "functional_1"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
input_1 (InputLayer)                                  [(None, 180, 360, 1)]                           0                 
________________________________________________________________________________________________________________________
resize_layer (ResizeLayer)                            (None, 176, 360, 1)                             0                 
________________________________________________________________________________________________________________________
periodic_padding2d (PeriodicPadding2D)                (None, 192, 376, 1)                             0                 
________________________________________________________________________________________________________________________
conv2d (Co

In [27]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [ ]:
history = autoencoder.fit(x=data_train, epochs=20)

Epoch 1/20
360/360 [==============================] - 428s 1s/step - loss: 1.9217
Epoch 2/20
360/360 [==============================] - 428s 1s/step - loss: 1.8423
Epoch 3/20
360/360 [==============================] - 430s 1s/step - loss: 1.7441
Epoch 4/20
360/360 [==============================] - 435s 1s/step - loss: 1.6198
Epoch 5/20
360/360 [==============================] - 429s 1s/step - loss: 1.4690
Epoch 6/20
360/360 [==============================] - 428s 1s/step - loss: 1.2701
Epoch 7/20
360/360 [==============================] - 423s 1s/step - loss: 0.7998
Epoch 8/20
360/360 [==============================] - 422s 1s/step - loss: 0.5070
Epoch 9/20
360/360 [==============================] - 422s 1s/step - loss: 0.4683
Epoch 10/20
360/360 [==============================] - 422s 1s/step - loss: 0.4364
Epoch 11/20
360/360 [==============================] - 422s 1s/step - loss: 0.4086
Epoch 12/20
201/360 [===============>..............] - ETA: 3:06 - loss: 0.3854